# 📁 專案資料夾清理與整理工具

這個 notebook 用於重新整理貸款審批專案資料夾的內容，識別並處理不必要的檔案。

## 🎯 目標
- 掃描專案資料夫結構
- 識別不必要的檔案（臨時檔案、快取檔案、重複文檔等）
- 生成清理報告
- 安全地清理和整理檔案

## 📋 處理流程
1. 匯入必要的函式庫
2. 掃描專案資料夾結構  
3. 定義檔案分類規則
4. 識別不必要的檔案
5. 建立清理報告
6. 執行檔案清理操作

---

In [ ]:
# 匯入必要的函式庫
import os
import shutil
import pathlib
import glob
import datetime
from pathlib import Path
import pandas as pd
from typing import List, Dict, Tuple

# 設定專案根目錄
PROJECT_ROOT = Path("C:/Users/johnn/Loan-approval")
print(f"專案根目錄: {PROJECT_ROOT}")
print(f"目錄存在: {PROJECT_ROOT.exists()}")

# 建立目錄結構分析結果的資料夾
CLEANUP_OUTPUT = PROJECT_ROOT / "notebooks" / "cleanup_output"
CLEANUP_OUTPUT.mkdir(exist_ok=True)
print(f"清理輸出目錄: {CLEANUP_OUTPUT}")

In [ ]:
# 掃描專案資料夾結構
def scan_project_structure(root_path: Path) -> Dict[str, any]:
    """
    掃描專案資料夾結構，返回詳細資訊
    """
    structure = {
        'directories': [],
        'files': [],
        'total_size': 0,
        'file_counts_by_ext': {},
        'large_files': []  # 大於 50MB 的檔案
    }
    
    for root, dirs, files in os.walk(root_path):
        root_path_obj = Path(root)
        
        # 記錄目錄
        relative_path = root_path_obj.relative_to(PROJECT_ROOT)
        structure['directories'].append(str(relative_path))
        
        # 記錄檔案
        for file in files:
            file_path = root_path_obj / file
            try:
                file_size = file_path.stat().st_size
                file_ext = file_path.suffix.lower()
                
                file_info = {
                    'path': str(file_path.relative_to(PROJECT_ROOT)),
                    'name': file,
                    'size': file_size,
                    'extension': file_ext,
                    'modified': datetime.datetime.fromtimestamp(file_path.stat().st_mtime)
                }
                
                structure['files'].append(file_info)
                structure['total_size'] += file_size
                
                # 統計副檔名
                if file_ext in structure['file_counts_by_ext']:
                    structure['file_counts_by_ext'][file_ext] += 1
                else:
                    structure['file_counts_by_ext'][file_ext] = 1
                
                # 記錄大檔案
                if file_size > 50 * 1024 * 1024:  # 50MB
                    structure['large_files'].append(file_info)
                    
            except (OSError, PermissionError) as e:
                print(f"無法存取檔案 {file_path}: {e}")
                
    return structure

# 執行掃描
print("🔍 掃描專案結構中...")
project_structure = scan_project_structure(PROJECT_ROOT)

print(f"📊 掃描結果:")
print(f"  - 總目錄數: {len(project_structure['directories'])}")
print(f"  - 總檔案數: {len(project_structure['files'])}")
print(f"  - 總大小: {project_structure['total_size'] / (1024*1024):.2f} MB")
print(f"  - 大檔案數 (>50MB): {len(project_structure['large_files'])}")

# 顯示副檔名統計
print(f"\n📁 檔案類型統計:")
for ext, count in sorted(project_structure['file_counts_by_ext'].items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"  {ext if ext else '無副檔名'}: {count} 個檔案")